In [0]:
%sql
create or refresh streaming table bronze_customers
comment 'raw customers data ingested from the source system to operational data'
tblproperties ('quality' = 'bronze')
AS 
select *,
        current_timestamp() as ingestion_ts,
        _metadata.file_path as input_file_path
from cloud_files(
  '/Volumes/circuitbox/landing/operational_data/customers/', 'json',
  map('cloudfiles.inferColumnTypes', 'true')
);

Name,Type
created_date,string
customer_id,bigint
customer_name,string
date_of_birth,string
email,string
telephone,string
_rescued_data,string
ingestion_ts,timestamp
input_file_path,string


#To silver Table

--clean customer silver

In [0]:
create or refresh streaming table silver_customers_clean (
     
    constraint valid_customer_id EXPECT (customer_id is not null) on violation fail update,
    constraint valid_customer_name EXPECT (customer_name is not null) on violation drop row,
    constraint valid_telephone EXPECT (LENGTH(telephone) >= 10),
    constraint valid_email EXPECT (email is not null),
    constraint valid_date_of_birth expect (date_of_birth >= '1920-01-01')

)

comment 'Cleaned customers data'
tblproperties ('quality' = 'silver')
As 
select customer_id,
       customer_name,
       Cast (date_of_birth as date) as date_of_birth,
       telephone,
       email,
       cast(created_date as date) as created_date
    from stream(live.bronze_customers) 

Name,Type
customer_id,bigint
customer_name,string
date_of_birth,date
telephone,string
email,string
created_date,date


##from clean_customer_silver to Customer_Silver

In [0]:
create or refresh streaming table silver_customers
comment 'scd type 1 customers data'
tblproperties ('quality' = 'silver')

In [0]:
apply changes into live.silver_customers
from stream (live.silver_customers_clean)
keys (customer_id)
sequence by created_date
stored as scd type 1;